Experiments with the EPA database fetching data using their REST api: https://aqs.epa.gov/aqsweb/documents/data_api.html

The above site lists everthing that can be done, start with registering your email so that you get a key to use. The code below parameterizes both, fill in the appropriate values before starting.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import requests

In [ ]:
email = "me@email.com"
key   = "my key"

In the subsequent code, I follow a fairly standard approach. 

* Construct a request
* Convert the response into a json table
* Extract the data, convert into a pandas dataframe

After this, what we do depends on what we've requested.

In [ ]:
## What parameter classes are being tracked?
response = requests.get(f'https://aqs.epa.gov/data/api/list/classes?email={email}&key={key}')
js = response.json()
params = pd.DataFrame(js['Data'])
print(params)

In [ ]:
## parameters in a class -- taking AQI POLLUTANTS for this
response = requests.get(f'https://aqs.epa.gov/data/api/list/parametersByClass?email={email}&key={key}&pc=AQI POLLUTANTS')
js = response.json()
aqi = pd.DataFrame(js['Data'])
print(aqi)

In [ ]:
## Get a list of states so that we can use the state id -- left as an exercise for the reader!

In [ ]:
## Get counties for California:
response = requests.get(f'https://aqs.epa.gov/data/api/list/countiesByState?email={email}&key={key}&state=06')
js = response.json()
counties = pd.DataFrame(js['Data'])
print(counties)

According to Google, Bakersfield, CA is the most polluted city in the US. It's located in Kern County, 
so now we have

State code 06
County code 029

Let's see what monitoring stations we have for Kern county!

In [ ]:
response = requests.get(f'https://aqs.epa.gov/data/api/list/sitesByCounty?email={email}&key={key}&state=06&county=029')
js = response.json()
sites = pd.DataFrame(js['Data'])
print(sites.shape)

In [ ]:
print(sites)

In [ ]:
# now lets get some data for one site in 2018
response = requests.get(f'https://aqs.epa.gov/data/api/sampleData/bySite?email={email}&key={key}&param=42101&bdate=20180101&edate=20181231&state=06&county=029&site=2012')
js = response.json()
data = pd.DataFrame(js['Data'])
print(data.shape)

In [ ]:
print(data.columns)

In [ ]:
d_sorted = data.sort_values(['date_gmt', 'time_gmt'])

In [ ]:
d_sorted.plot(x='date_gmt', y='sample_measurement')

In [ ]:
## Bangor, Maine is the least polluted city in the US. so....
## Get counties for Maine:
response = requests.get(f'https://aqs.epa.gov/data/api/list/countiesByState?email={email}&key={key}&state=23')
js = response.json()
counties = pd.DataFrame(js['Data'])
print(counties)

In [ ]:
## Bangor is in Penobscot county: 019
response = requests.get(f'https://aqs.epa.gov/data/api/list/sitesByCounty?email={email}&key={key}&state=23&county=019')
js = response.json()
sites = pd.DataFrame(js['Data'])
print(sites.shape)


In [ ]:
print(sites)

In [ ]:
pd.set_option('display.max_rows', None)
print(sites)

In [ ]:
# now lets get some data for one site in 2018
response = requests.get(f'https://aqs.epa.gov/data/api/sampleData/bySite?email={email}&key={key}&param=42101&bdate=20180101&edate=20181231&state=23&county=019&site=4005')
js = response.json()
data = pd.DataFrame(js['Data'])
print(data.shape)

Not all values returned have valid data -- or any data! The following loop is useful to figure out which sites (in this case) have data. Of course, there are better & more efficient ways of doing this!

In [ ]:
for i in sites['code']:
    response = requests.get(f'https://aqs.epa.gov/data/api/sampleData/bySite?email={email}&key={key}&param=42101&bdate=20180101&edate=20181231&state=23&county=019&site={i}')
    js = response.json()
    data = pd.DataFrame(js['Data'])
    print(response, i, data.shape)
    

In [ ]:
print(response)
print(js)

okay, so that sucks!!

Lets try... South Burlington, Vt!

In [ ]:
## Get counties for Vermont:
response = requests.get(f'https://aqs.epa.gov/data/api/list/countiesByState?email={email}&key={key}&state=50')
js = response.json()
counties = pd.DataFrame(js['Data'])
print(counties)

In [ ]:
#south Burlington is in Chittenden county 007
response = requests.get(f'https://aqs.epa.gov/data/api/list/sitesByCounty?email={email}&key={key}&state=50&county=007')
js = response.json()
sites = pd.DataFrame(js['Data'])
print(sites.shape)


In [ ]:
for i in sites['code']:
    response = requests.get(f'https://aqs.epa.gov/data/api/sampleData/bySite?email={email}&key={key}&param=42101&bdate=20180101&edate=20181231&state=50&county=007&site={i}')
    js = response.json()
    data = pd.DataFrame(js['Data'])
    print(response, i, data.shape)
    

In [ ]:
## Success! sites 0007 and 0014 have data!
# now lets get some data for one site in 2018
response = requests.get(f'https://aqs.epa.gov/data/api/sampleData/bySite?email={email}&key={key}&param=42101&bdate=20180101&edate=20181231&state=50&county=007&site=0014')
js = response.json()
data_vt = pd.DataFrame(js['Data'])
print(data_vt.shape)

The remainder of this notebook is basically a step-by-step demonstration of getting to the plot that I want, from the initial attempt

In [ ]:
dvt_sorted = data_vt.sort_values(['date_gmt', 'time_gmt'])

In [ ]:
dvt_sorted.plot(x='date_gmt', y='sample_measurement')

In [ ]:
fig, ax = plt.subplots(1, 2)
ax[0].plot(d_sorted['date_gmt'], d_sorted['sample_measurement'])
ax[1].plot(dvt_sorted['date_gmt'], dvt_sorted['sample_measurement'])
plt.show()

In [ ]:
## okay, let's make the figures a bit larger
fig, ax = plt.subplots(1, 2, figsize=(16,8))
ax[0].plot(d_sorted['date_gmt'], d_sorted['sample_measurement'])
ax[1].plot(dvt_sorted['date_gmt'], dvt_sorted['sample_measurement'])
plt.show()

In [ ]:
## ...and get the axes lined up!
fig, ax = plt.subplots(1, 2, figsize=(16,8), sharey=True)
ax[0].plot(d_sorted['date_gmt'], d_sorted['sample_measurement'])
ax[1].plot(dvt_sorted['date_gmt'], dvt_sorted['sample_measurement'])
plt.show()

In [ ]:
# let's add a few descriptors, so we know what's what
## ...and get the axes lined up!
fig, ax = plt.subplots(1, 2, figsize=(16,8), sharey=True)
fig.suptitle('CO2 difference between the most and least polluted cities in the USA')
ax[0].plot(d_sorted['date_gmt'], d_sorted['sample_measurement'])
ax[0].set(title = 'Bakersfield, CA', xlabel = '2018', ylabel = 'CO')

ax[1].plot(dvt_sorted['date_gmt'], dvt_sorted['sample_measurement'])
ax[1].set(title = 'South Burlington, VT', xlabel = '2018', ylabel = 'CO')
plt.show()

Finally(?), let's fix the number of ticks we see on the x-axis

In [ ]:
print(d_sorted.dtypes)
d_sorted['date_local'] = pd.to_datetime(d_sorted['date_local'])
print(d_sorted.dtypes)

In [ ]:
print(dvt_sorted.dtypes)
dvt_sorted['date_local'] = pd.to_datetime(dvt_sorted['date_local'])
print(dvt_sorted.dtypes)

In [ ]:

fig, ax = plt.subplots(1, 2, figsize=(16,8), sharey=True)
fig.suptitle('CO2 difference between the most and least polluted cities in the USA')
ax[0].plot(d_sorted['date_local'], d_sorted['sample_measurement'])
ax[0].set(title = 'Bakersfield, CA', xlabel = '2018', ylabel = 'CO')

ax[1].plot(dvt_sorted['date_local'], dvt_sorted['sample_measurement'])
ax[1].set(title = 'South Burlington, VT', xlabel = '2018', ylabel = 'CO')
plt.show()

Now that the overall structure is in place, we can do the same thing for the other pollutants:
* SO_2
* NO_2
* O_3
* PM_{10}
* PM_{2.5}
* Acceptable PM2.5 AQI & Speciation Mass, whateverthatis!!